In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_10701/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 02:37:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/18 02:37:04 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Matches table
matches_data = {
    'player_id': [1, 1, 1, 1, 1, 2, 2, 3],
    'match_day': ['2022-01-17', '2022-01-18', '2022-01-25', '2022-01-31', '2022-02-08', '2022-02-06', '2022-02-08', '2022-03-30'],
    'result': ['Win', 'Win', 'Win', 'Draw', 'Win', 'Lose', 'Lose', 'Win']
}

# Create DataFrame
matches_df = pd.DataFrame(matches_data)

# Display the DataFrame
print("Matches table:")
print(matches_df)



df_person = spark.createDataFrame(matches_df)
df_person.createOrReplaceTempView("Matches")



Matches table:
   player_id   match_day result
0          1  2022-01-17    Win
1          1  2022-01-18    Win
2          1  2022-01-25    Win
3          1  2022-01-31   Draw
4          1  2022-02-08    Win
5          2  2022-02-06   Lose
6          2  2022-02-08   Lose
7          3  2022-03-30    Win


In [6]:
query = """
    select
        *,
        sum(
            case
                when result = 'Win' then 0
                else 1
            end
        ) over (partition by player_id order by match_day asc) as group_id
    from Matches m

"""

In [7]:
spark.sql(query).show()

+---------+----------+------+--------+
|player_id| match_day|result|group_id|
+---------+----------+------+--------+
|        1|2022-01-17|   Win|       0|
|        1|2022-01-18|   Win|       0|
|        1|2022-01-25|   Win|       0|
|        1|2022-01-31|  Draw|       1|
|        1|2022-02-08|   Win|       1|
|        2|2022-02-06|  Lose|       1|
|        2|2022-02-08|  Lose|       2|
|        3|2022-03-30|   Win|       0|
+---------+----------+------+--------+



In [14]:
query_result = f"""
    select
        t.player_id,
        ifnull(k.longest_streak, 0)
    from (
        select distinct player_id from Matches m order by player_id asc
    ) t left join (
        select
            player_id,
            max(total_win) as longest_streak
        from (
            select
                player_id, 
                group_id, 
                count(*) as total_win
            from (
                {query}
            ) where result = 'Win'
            group by player_id, group_id
        ) group by player_id
    ) k on t.player_id = k.player_id
    order by t.player_id asc

"""

In [15]:
spark.sql(query_result).show()

+---------+-------------------------+
|player_id|ifnull(longest_streak, 0)|
+---------+-------------------------+
|        1|                        3|
|        2|                        0|
|        3|                        1|
+---------+-------------------------+

